In [1]:
import datetime
import pandas as pd
import requests
import json

df=pd.read_csv("../coronavirus_2021q1_all.csv")
df["Publish Date"]=df["Publish Date"].apply(lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")))
df=df[df["Publish Date"]>datetime.datetime.timestamp(datetime.datetime(2021, 3, 15, 0))]
# print(f"{len(pd.unique(df['Author']))} Unique users")

In [2]:
value_counts=df["Author"].value_counts()
print(value_counts)
df=df[~df["Author"].isin(value_counts[value_counts>=2000].index)]

[deleted]               1999
AutoModerator           1010
lordhamster1977          286
JenniferColeRhuk         194
ThatsJustUn-American     149
                        ... 
amr500gvrd                 1
ox_raider                  1
cheffromspace              1
sum8fever                  1
BaltSuz                    1
Name: Author, Length: 10518, dtype: int64


In [3]:
from time import sleep, time

from requests import Timeout

next = time()+2
def parse_comment(comment):
    body = comment['body'].replace('\n',' ').replace('\t',' ').replace('\r',' ').replace('\r\n',' ')
    author = comment['author']
    com_id = comment['id']
    score = comment['score']
    created = datetime.datetime.fromtimestamp(comment['created_utc']) 
    permalink = comment['permalink']
    subreddit = comment['subreddit']
    parent_id = comment['parent_id']
    post_id=comment['link_id']
    return (com_id,subreddit,body,author,score,created,permalink,parent_id,post_id)

def search_user_comments(user, i, limit:int, before:int, after:int):
    global next
    user_comments=[]
    sleep(max(0,next-time()))
    next = time()+2
    while True:
        sleep(max(0,next-time()))
        next = time()+2
        url= f"https://api.pushshift.io/reddit/search/comment?limit=1000&before={before}&after={after}{' ' if user is None else '&author='+user}"

        while True:
            try:
                r = requests.get(url, timeout=5)
                data = json.loads(r.text)['data']
                break
            except Timeout as t:
                print(f"Timed Out on user {user} ({i}): {t}", end='\r')
                sleep(5)
            except Exception as e:
                print(f"Errored on user {user} ({i}): {e}", end='\r')
                sleep(5)
        
        if len(data)==0:
            break
        
        user_comments+=data
        if len(user_comments)>=limit:
            user_comments=user_comments[:limit]
            break

        before=user_comments[-1]["created_utc"]-1

    parsed_comments=[]
    for comment in user_comments:
        if comment['subreddit']!="Coronavirus":
            parsed_comments.append(parse_comment(comment))
    
    return parsed_comments


In [4]:
before = int(datetime.datetime(2021, 3, 30, 0).timestamp())
after = int(datetime.datetime(2020, 10, 1, 0).timestamp())

comms = {}

for i,user in enumerate(pd.unique(df["Author"])):
     usercomms=search_user_comments(user,i,1000,before,after)
     print(f"Gathered {len(usercomms)} comments for user {user} ({i}/{len(pd.unique(df['Author']))}) \t\t ",end='\r')
     comms[user]=usercomms

In [6]:
import time
import csv
def updateSubs_file():
    upload_count = 0
    print("input filename of submission file, please add .csv")
    filename = input()
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Subreddit","Body","Author","Score","Publish Date","Permalink","Parent_id","Post_id"]
        a.writerow(headers)
        for user in comms:
            a.writerows(comms[user])
            upload_count+=len(comms[user])
            
        print(str(upload_count) + " submissions have been uploaded")

updateSubs_file()

input filename of submission file, please add .csv
8667514 submissions have been uploaded
